# 1. Simulate the data

This notebook contains the code to simulate the data for the experiments. 

In [34]:
import pandas as pd
import numpy as np

In [43]:
### This is a placeholder code till we have code from Matt/Dan

np.random.seed(100)
true_scores = np.random.normal(2.692, 0.691, 10000)
df = pd.DataFrame({'id': ['id_{}'.format(i) for i in range(10000)],
                  'true': true_scores})
raters = []

for i in range(0, 1000):
    error = np.random.choice([0.1, 0.5, 1])
    rater_error = np.random.normal(0,error,10000)
    df['h_{}'.format(i)] = np.round(true_scores+rater_error)
    df_rater = pd.DataFrame({'rater_id': ['h_{}'.format(i)],
                            'error': error})
    raters.append(df_rater)

df_raters = pd.concat(raters)

systems = []

for j in range(0, 10):
    error = np.random.choice([0.1, 0.5, 1])
    system_error = np.random.normal(0,error,10000)
    df['sys_{}'.format(j)] = true_scores+system_error
    df_rater = pd.DataFrame({'system_id': ['sys_{}'.format(j)],
                            'error': error})
    systems.append(df_rater)

df_systems = pd.concat(systems)

In [45]:
df.to_csv('../data/data.csv', index=False)
df_raters.to_csv('../data/raters.csv', index=False)
df_systems.to_csv('../data/systems.csv', index=False)